In [2]:
import osmnx as ox
import networkx as nx
import folium

In [4]:
import osmnx as ox
import networkx as nx
import folium

# Define the individual places in Bicol Region
places = ["Naga City, Philippines"]

# Coordinates of Barangay Halls
brgy = [
    {"province": "Camarines Sur", "City": "Naga", "barangay": "Cararayan", "lat": 13.628834653694073, "lon": 123.24006454110972}
]

# Create an empty directed multigraph to combine all places
combined_graph = nx.MultiDiGraph()

# Loop through each place and retrieve the graph for each
for place in places:
    graph = ox.graph_from_place(place, custom_filter='["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|ferry"]', network_type='drive')
    combined_graph = nx.compose(combined_graph, graph)

# Get the centroid of the combined places to initialize the map
centroid = ox.geocode_to_gdf(places[0]).unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=8, tiles='OpenStreetMap')

# Find the nearest nodes for each Barangay Hall and simulate damage
for barangay in brgy:
    nearest_node = ox.distance.nearest_nodes(combined_graph, barangay['lon'], barangay['lat'])

    # Mark this node as "damaged" on the map before removing it
    folium.Marker(
        location=[barangay['lat'], barangay['lon']],
        popup=f"{barangay['barangay']} - Damaged",
        icon=folium.Icon(color='red', icon='exclamation-sign')
    ).add_to(m)
    
    # Remove the node and its connected edges to simulate network damage
    if nearest_node in combined_graph:
        combined_graph.remove_node(nearest_node)

# Get the edges and nodes GeoDataFrames after node removal
gdf_nodes, gdf_edges = ox.graph_to_gdfs(combined_graph, nodes=True, edges=True)

# Add the remaining edges (roads and sea routes) to the map
for _, edge in gdf_edges.iterrows():
    points = [(y, x) for x, y in edge['geometry'].coords]
    folium.PolyLine(points, color="darkred", weight=1.5, opacity=0.6).add_to(m)

# Add the remaining nodes (intersections) to the map
for _, node in gdf_nodes.iterrows():
    folium.CircleMarker(location=[node['y'], node['x']],
                        radius=3, color='blue', fill=True, fill_opacity=0.8).add_to(m)

m.save("damaged_network_with_markers.html")

TypeError: Nominatim could not geocode query 'Naga City, Philippines' to a geometry of type (Multi)Polygon